# Data import

data를 가져오는 부분

In [1]:
import csv
import numpy as np

with open('train.csv','r') as dest_f:
    data_iter = csv.reader(dest_f, 
                           delimiter = ',', 
                           quotechar = '"')
    data = [data for data in data_iter]
data_array = np.asarray(data, dtype = 'str')
print(data_array)
#print(data_array.shape)

[['PassengerId' 'Survived' 'Pclass' ... 'Fare' 'Cabin' 'Embarked']
 ['1' '0' '3' ... '7.25' '' 'S']
 ['2' '1' '1' ... '71.2833' 'C85' 'C']
 ...
 ['889' '0' '3' ... '23.45' '' 'S']
 ['890' '1' '1' ... '30' 'C148' 'C']
 ['891' '0' '3' ... '7.75' '' 'Q']]


data의 한글부분을 수치화한다.
(남자는 0, 여자는 1로)

In [2]:
quantifi_data = data_array[1:,:]

for i in range(quantifi_data.shape[0]):
    if quantifi_data[i][4]=='male':
        quantifi_data[i][4]=float(0)
    elif quantifi_data[i][4]=='female':
        quantifi_data[i][4]=float(1)

### 나이 데이터의 빈 공간을 채우는 방법을 찾아보자

방법 1. 1912년 미국의 기대수명을 이용한다. 남자의 경우 51.5살, 여자의 경우 55.9살이라는 데이터를 이용한다.
###### http://www.demog.berkeley.edu/~andrew/1918/figure2.html

In [3]:
for i in range(quantifi_data.shape[0]):
    if quantifi_data[i][5]=='':
        if quantifi_data[i][4]:
            quantifi_data[i][5]=float(51.5)
        else:
            quantifi_data[i][5]=float(55.9)
            
print(quantifi_data[5])

['6' '0' '3' 'Moran, Mr. James' '0.0' '51.5' '0' '0' '330877' '8.4583' ''
 'Q']


test 할 데이터 또한 받아온후 동일 작업을 시행한다

In [4]:
with open('test.csv','r') as dest_f:
    data_iter1 = csv.reader(dest_f, 
                           delimiter = ',', 
                           quotechar = '"')
    data1 = [data1 for data1 in data_iter1]
data_array1 = np.asarray(data1, dtype = 'str')
print(data_array1)
#print(data_array.shape)

quantifi_data1 = data_array1[1:,:]

for i in range(quantifi_data1.shape[0]):
    if quantifi_data1[i][3]=='male':
        quantifi_data1[i][3]=0
    elif quantifi_data1[i][3]=='female':
        quantifi_data1[i][3]=1
        
for i in range(quantifi_data1.shape[0]):
    if quantifi_data1[i][4]=='':
        if quantifi_data1[i][3]:
            quantifi_data1[i][4]=float(51.5)
        else:
            quantifi_data1[i][4]=float(55.9)

[['PassengerId' 'Pclass' 'Name' ... 'Fare' 'Cabin' 'Embarked']
 ['892' '3' 'Kelly, Mr. James' ... '7.8292' '' 'Q']
 ['893' '3' 'Wilkes, Mrs. James (Ellen Needs)' ... '7' '' 'S']
 ...
 ['1307' '3' 'Saether, Mr. Simon Sivertsen' ... '7.25' '' 'S']
 ['1308' '3' 'Ware, Mr. Frederick' ... '8.05' '' 'S']
 ['1309' '3' 'Peter, Master. Michael J' ... '22.3583' '' 'C']]


각 데이터마다 모두 범위가 차이나므로 격차가 발생, nomalization이 필요하다.

In [5]:
def MinMaxScaler(data):
    numerator = data.astype(np.float32) - np.min(data.astype(np.float32), 0)
    denominator = np.max(data.astype(np.float32), 0) - np.min(data.astype(np.float32), 0)
    
    return numerator / (denominator + 1e-7)

In [6]:
quantifi_data[:,2] = MinMaxScaler(quantifi_data[:,2])
quantifi_data[:,4] = MinMaxScaler(quantifi_data[:,4])
quantifi_data[:,5] = MinMaxScaler(quantifi_data[:,5])
quantifi_data[:,6] = MinMaxScaler(quantifi_data[:,6])
quantifi_data1[:,1] = MinMaxScaler(quantifi_data1[:,1])
quantifi_data1[:,3] = MinMaxScaler(quantifi_data1[:,3])
quantifi_data1[:,4] = MinMaxScaler(quantifi_data1[:,4])
quantifi_data1[:,5] = MinMaxScaler(quantifi_data1[:,5])

#### 성별과 동승자수, Pclass, Parch, 나이로 분류해보자
#### x data size는 [datanum, 5(Pclass, 성별, 나이, 동승자수, Parch)]
#### y data size는 [datanum, 1]이다

In [7]:
import tensorflow as tf

x_data = np.vstack([quantifi_data[:,2], quantifi_data[:,4], quantifi_data[:,5], quantifi_data[:,6], quantifi_data[:,7]]).T
y_data = quantifi_data[:,1].reshape(quantifi_data.shape[0],1)
x_test_data = np.vstack([quantifi_data1[:,1], quantifi_data1[:,3], quantifi_data1[:,4], quantifi_data1[:,5], quantifi_data1[:,6]]).T
test_prediction = np.zeros([quantifi_data1.shape[0],1])

X = tf.placeholder(tf.float32, [None, 5])
Y = tf.placeholder(tf.float32, [None, 1])

W1 = tf.Variable(tf.random_normal([5,5]), name='weight1')
b1 = tf.Variable(tf.random_normal([5]), name='bias1')
layer1 = tf.sigmoid(tf.matmul(X, W1) + b1)

W2 = tf.Variable(tf.random_normal([5,5]), name='weight2')
b2 = tf.Variable(tf.random_normal([5]), name='bias2')
layer2 = tf.sigmoid(tf.matmul(layer1, W2) + b2)

W3 = tf.Variable(tf.random_normal([5,3]), name='weight3')
b3 = tf.Variable(tf.random_normal([3]), name='bias3')
layer3 = tf.sigmoid(tf.matmul(layer2, W3) + b3)

W4 = tf.Variable(tf.random_normal([3,1]), name='weight4')
b4 = tf.Variable(tf.random_normal([1]), name='bias4')
hypothesis = tf.sigmoid(tf.matmul(layer3, W4) + b4)

cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis))
#train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)
train = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y),dtype=tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(10001):
        sess.run(train, feed_dict={X: x_data,Y: y_data})
        
        if step % 1000 == 0:
            print(step, sess.run(cost, feed_dict={X: x_data,Y: y_data}), sess.run([W1, W2, W3]))
    
    h, c, a = sess.run([hypothesis, predicted, accuracy], feed_dict={X: x_data,Y: y_data})
    print("\nHypothesis: ", h, "\nCorrect: ", c, "\nAccuracy: ", a)
    
    #이제 실제 데이터를 집어넣어 예측해보자
    test_prediction = sess.run([hypothesis], feed_dict={X: x_test_data})
    print(test_prediction)
    

0 0.6867516 [array([[ 1.6810081 , -0.5057388 ,  0.42184472,  0.18038568, -0.0227443 ],
       [ 1.068124  , -0.48234448, -0.48247367, -0.21845964,  1.2789049 ],
       [-0.20068757, -0.97939163, -0.84187293,  0.08976731, -0.3198118 ],
       [ 0.05090017, -0.5212802 , -0.7188569 ,  0.51794493, -0.48494047],
       [-0.19765778,  0.9898206 ,  1.8248636 ,  0.81733394,  1.7009047 ]],
      dtype=float32), array([[-1.7984909 , -0.10936839,  0.18755966,  0.17230543,  0.22189349],
       [ 0.0070378 ,  0.2684911 , -0.48843476, -0.23129058,  0.60271376],
       [-0.41636497,  0.31903663,  0.51287377, -0.8579737 , -1.5563788 ],
       [-0.82407206,  0.70261174, -0.77823967, -0.06151137, -0.05238742],
       [-0.36580658, -1.999488  ,  0.09187845,  0.3886262 ,  0.70287704]],
      dtype=float32), array([[-0.14195262, -1.1695154 , -0.2835006 ],
       [ 0.5224298 , -0.21419676,  0.2209278 ],
       [ 2.1697748 ,  1.1585444 , -0.12408004],
       [-3.0300972 , -0.76849085, -0.11709318],
       [ 

8000 0.3306834 [array([[ -4.639254  ,  -1.0892745 ,   2.146368  ,   5.5821457 ,
         -4.17741   ],
       [  7.609686  ,  -0.16670412,   1.5760072 ,   0.05046359,
          8.606319  ],
       [  1.6343787 , -11.570147  ,  -1.9240181 , -11.791464  ,
         -5.666116  ],
       [ -2.713154  , -11.245236  ,  20.952894  ,  18.231771  ,
        -22.293472  ],
       [ -0.8810261 ,  -0.05633998,   1.4295256 ,   2.4594858 ,
         -2.9176795 ]], dtype=float32), array([[ -9.538817 ,   1.0051334,   7.680486 ,  -7.957534 ,  16.7645   ],
       [ -5.8824515,  -6.4598613,  19.99012  ,  -9.993794 ,   9.447408 ],
       [ -5.8936343,   3.6906776,   3.7271924,  -2.289581 ,   5.752886 ],
       [  9.170264 ,   3.9242532,  -3.5988517,   3.3629396,  -6.6761165],
       [  7.0087667,   6.076675 ,   3.2098246,  -3.5845966, -18.010489 ]],
      dtype=float32), array([[-16.754824  ,  -2.6308234 , -17.802078  ],
       [  1.1989688 ,  -2.6322262 ,  -0.46152368],
       [ 15.321714  ,   3.3545365 ,  

test_prediction의 값을 0과 1로 정규화 시켜준다

In [8]:
for i in range(len(test_prediction[0])):
    if test_prediction[0][i] > 0.5:
        test_prediction[0][i] = 1
    else:
        test_prediction[0][i] = 0

print(test_prediction)

[array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.]

예측값들을 csv파일로 저장한다

In [9]:
f = open('prediction4.csv', 'w', encoding='utf-8', newline='')
wr = csv.writer(f)
wr.writerow(["PassengerId", "Survived"])
for i in range(len(test_prediction[0])):
    wr.writerow([892+i, int(test_prediction[0][i])])
    
f.close()